In [1]:
import os
import sys

current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, "../eval", ".."))
sys.path.append(parent_parent_dir)

from state_control.controllable_rwkv import VectorControlRWKV, ContrastControlRWKV
import ipywidgets as widgets
from IPython.display import display

Using /root/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py38_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


ninja: no work to do.


In [2]:
MODEL_PATH = r'../../models/RWKV-5-World-3B-v2-20231113-ctx4096'

In [3]:
model = VectorControlRWKV(MODEL_PATH)

RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading ../../models/RWKV-5-World-3B-v2-20231113-ctx4096.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, float16], store 33 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-float16-float16 28-cuda-float16-float16 29-cuda-float16-float16 30-cuda-float16-float16 31-cuda-float16-float16 32-cuda-float16-float16 
emb.weight                    

Using /root/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py38_cu118/rwkv5/build.ninja...
Building extension module rwkv5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module rwkv5...


ninja: no work to do.


### Language Control Demo
##### Here, we demonstrate the model's capability to control the generated language (Chinese/English) using directional vectors.

First, we construct pairs of prompt words that have the same meaning but are in different languages, providing the model with directions towards various languages.

In [99]:
pos_str_list = ['这是一个句子', '这句话可以是没有任何意义的']
neg_str_list = ['Here is a sentence', 'This sentence can be meaningless.']

Tokenize the sentences and then update the control vectors.

In [100]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

We create a slider to easily adjust the value of the control factor.
You can adjust the slider to see how the control effect changes.

In [101]:
language_control_slider = widgets.FloatSlider(
    value=0,
    min=-1.2,
    max=1.5,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(language_control_slider)

FloatSlider(value=0.0, description='Coeff:', max=1.5, min=-1.2, step=0.01)

In [104]:
test_input = "\nHere is a story:"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=language_control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

 一个小男孩，他的名字叫做小明，他非常喜欢玩游戏。他的父母都是游戏迷，他们经常带他去游戏厅玩游戏。小


You can try both from English to Chinese and from Chinese to English

In [8]:
test_input = "\n下面是一个儿童故事\n"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=language_control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

一只小猫咪
在一个小村庄里
它喜欢玩耍
和小朋友们一起
它们一起玩耍
在草地上奔跑
在树林里追逐
小


### output format control
##### Next, we will show how to manipulate the model's output format to output content in Python list.

In [105]:
neg_str_list = [
    "The cat sat on the mat.",
    "It is raining heavily outside.",
    "Learning new languages can be fun and challenging.",
    "Space exploration offers great opportunities for scientific advancement.",
    "Healthy eating contributes to overall wellness."
]

pos_str_list = [
    "['The', 'cat', 'sat', 'on', 'the', 'mat.']",
    "['It', 'is', 'raining', 'heavily', 'outside.']",
    "['Learning', 'new', 'languages', 'can', 'be', 'fun', 'and', 'challenging.']",
    "['Space', 'exploration', 'offers', 'great', 'opportunities', 'for', 'scientific', 'advancement.']",
    "['Healthy', 'eating', 'contributes', 'to', 'overall', 'wellness.']"
]

In [106]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

In [115]:
control_slider = widgets.FloatSlider(
    value=0,
    min=0.,
    max=2.0,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(control_slider)

FloatSlider(value=0.0, description='Coeff:', max=2.0, step=0.01)

Adjust the slider to see how the control effect changes.

In [117]:
test_input = "\nUser: List 10 well-known physicists.\n\nAssistant:"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

 ['Albert', 'Einstein', 'Isaac', 'Newton', 'Max', 'Planck', 'Albert', 'Schrödinger', 'Erwin', 'Friedrich', 'Heisenberg', 'Niels', 'Bo


### Control of Emotion in Model Output
##### Next, we show how we control the emotions in the model.

In [64]:
pos_str_list = [
    "Question: How was your day?\n\nAnswer: It was fantastic, full of exciting opportunities!",
    "Question: Did you enjoy your meal?\n\nAnswer: Absolutely, it was delicious and satisfying!",
    "Question: Are you looking forward to the weekend?\n\nAnswer: Definitely, I have so many fun plans!",
    "Question: How do you feel about your job?\n\nAnswer: I love it, it's challenging and rewarding!",
    "Question: Did you like the movie?\n\nAnswer: Yes, it was entertaining and thought-provoking!",
    "Question: How's the weather today?\n\nAnswer: It's lovely, sunny and warm!",
    "Question: Are you happy with your new phone?\n\nAnswer: Absolutely, it has amazing features!",
    "Question: Did you enjoy your vacation?\n\nAnswer: Yes, it was relaxing and fun!",
    "Question: How do you feel about the future?\n\nAnswer: I'm optimistic, there are so many possibilities!",
    "Question: Are you satisfied with your current living situation?\n\nAnswer: Yes, I'm very comfortable and content!"
]

neg_str_list = [
    "Question: How was your day?\n\nAnswer: It was terrible, nothing went right.",
    "Question: Did you enjoy your meal?\n\nAnswer: Not at all, it was bland and unappetizing.",
    "Question: Are you looking forward to the weekend?\n\nAnswer: Not really, I have nothing interesting to do.",
    "Question: How do you feel about your job?\n\nAnswer: I dislike it, it's stressful and unrewarding.",
    "Question: Did you like the movie?\n\nAnswer: No, it was boring and poorly made.",
    "Question: How's the weather today?\n\nAnswer: It's dreadful, cold and rainy.",
    "Question: Are you happy with your new phone?\n\nAnswer: Not at all, it's complicated and glitchy.",
    "Question: Did you enjoy your vacation?\n\nAnswer: No, it was stressful and unenjoyable.",
    "Question: How do you feel about the future?\n\nAnswer: I'm pessimistic, things seem uncertain and scary.",
    "Question: Are you satisfied with your current living situation?\n\nAnswer: No, I'm unhappy and looking for a change."
]

In [65]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

In [66]:
control_slider = widgets.FloatSlider(
    value=0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(control_slider)

FloatSlider(value=0.0, description='Coeff:', max=1.0, min=-1.0, step=0.01)

Adjust the slider to see how the control effect changes.

In [98]:
test_input = "\nMy mood"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

 is not good.
I'm not in the mood to be nice.
I'm not in the mood to be nice.
I'm not in the mood to be nice.
I'm not in the mood to be nice.
I
